In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/benjaminrike1/social_score_csr_reports.git

Cloning into 'social_score_csr_reports'...
remote: Enumerating objects: 456, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 456 (delta 15), reused 38 (delta 12), pack-reused 415
Receiving objects: 100% (456/456), 1.74 GiB | 37.24 MiB/s, done.
Resolving deltas: 100% (132/132), done.


In [ ]:
!pip3 install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 7.8 MB/s 


In [ ]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 6.7 MB/s 
     |████████████████████████████████| 511.7 MB 6.4 kB/s 
     |████████████████████████████████| 438 kB 59.1 MB/s 
     |████████████████████████████████| 5.8 MB 51.6 MB/s 
     |████████████████████████████████| 1.6 MB 54.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninst

In [ ]:
pip install -q tf-models-official==2.7.0

     |████████████████████████████████| 1.8 MB 8.1 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 596 kB 61.6 MB/s 
     |████████████████████████████████| 92 kB 10.4 MB/s 
     |████████████████████████████████| 352 kB 56.1 MB/s 
     |████████████████████████████████| 47.8 MB 44 kB/s 
     |████████████████████████████████| 1.1 MB 53.9 MB/s 
     |████████████████████████████████| 237 kB 54.2 MB/s 
     |████████████████████████████████| 1.2 MB 49.5 MB/s 
     |████████████████████████████████| 99 kB 7.4 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import pickle5 as pickle

In [ ]:
model_path = 'social_score_csr_reports/models/uten_augment/'
reloaded_model = tf.saved_model.load(model_path)


In [ ]:
def print_my_examples(inputs, results):
  print("Probabilities is on the format: neutral, positive, negative")
  result_for_printing = \
    [f'input: {inputs[i]:<30} : \n Probability: Signal: {results[i][0]:.2f}    Noise: {results[i][1]:.2f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    'In 2020, we continued to provide a framework of policies, standards, and implementation expectations to guide our supplier business relationships',
    'In 2020, we conducted risk assessments to help us identify and prioritize the most critical issue areas and suppliers. ',
    'We invest in charitable and community causes in all of our markets and our people regularly take part in volunteering activities to support social initiatives in our communities.',
    'As signatories of the Valuable 500 pledge, we are committed to putting disability on the business leadership agenda',
    'Our diversity was reduced this year',
    'Football is a sport',
    'In 2020, 35.6% of leadership positions were held by women, up from 35.5% in 2019. ',
    'Zero fatalities occurred during the year',
    'The launch of our cultural change CARE programme, which will embed ‘the seven health and safety habits’ within our organisation, was another success and', #positive
    '22% of our senior leadership team members are women', #negative?
    'Our Health and Safety Management System', #neutral
    'During the first quarter of 2021, we saw a spike in health and safety incidents',
    'Even though one uses generally the same letter “M” for both martingales and Markov process, these are a priori completely different processes!',
    'Not every process is either a sub- or a supermartingale',
    'This repository is for the exercises, homework and project. The course syllabus can be found here.',
    'Jeg heter Benjamin og skriver på norsk',
    'To establish the communication between the robot and the cloud via Wi-Fi, you need to find the IP address, as shown in Figure 3.',
    'We have drastically reduced our CO2 emission this year as we prepare to meet the latest regulatory demands.'
]
reloaded_results = tf.nn.softmax(reloaded_model(tf.constant(examples)))

print('Results from the model:')
print_my_examples(examples, reloaded_results)

Results from the model:
Probabilities is on the format: neutral, positive, negative
input: In 2020, we continued to provide a framework of policies, standards, and implementation expectations to guide our supplier business relationships : 
 Probability: Signal: 1.00    Noise: 0.00
input: In 2020, we conducted risk assessments to help us identify and prioritize the most critical issue areas and suppliers.  : 
 Probability: Signal: 1.00    Noise: 0.00
input: We invest in charitable and community causes in all of our markets and our people regularly take part in volunteering activities to support social initiatives in our communities. : 
 Probability: Signal: 1.00    Noise: 0.00
input: As signatories of the Valuable 500 pledge, we are committed to putting disability on the business leadership agenda : 
 Probability: Signal: 1.00    Noise: 0.00
input: Our diversity was reduced this year : 
 Probability: Signal: 1.00    Noise: 0.00
input: Football is a sport            : 
 Probability: Sign

In [ ]:
filename = 'social_score_csr_reports/all_reports_df.pickle'
with open(filename, 'rb') as fh:
  data = pickle.load(fh)
data.reset_index(inplace=True)
data.drop(labels=['Company'], inplace=True, axis=1)
data.rename({'index':'Company'}, axis=1, inplace=True)
data.head(5)

,Company,sentences_2016,sentences_2017,sentences_2018,sentences_2019,sentences_2020
0,3M,[2016 Sustainability Report “Starting with tec...,"[We are#improvinglives, 90,000 employees helpi...","[2018 , Sustainability Report, We tapped youth...","[Improving every life2019 , S...","[Improvingeverylife2020 , Sustainability Repor..."
1,Abbott,"[GLOBALSUSTAINABILITY REPORT 2016, FROM OUR C...","[2017 , GLOBAL , SUSTAINABILITY , REPORT, ABOU...","[GLOBAL SUSTAINABILITY REPORT 2018, CHANGING...","[2019 GLOBAL SUSTAINABILITY REPORTCONTENTS, Su...","[2020 GLOBAL SUSTAINABILIT Y REPORT, COVID-19:..."
2,Accenture,"[DIFFERENCE, CORPORATE, CITIZENSHIP, REPORT 20...","[CORPORATE, CITIZENSHIPREPORT 2017 , See how S...","[ IMPROVING , THE WAY , THE , WORLD WORKS A...","[BUILDING A FUTURE OF SHARED , SUCCESS, Corp...",[Building a Future of Shared SuccessCorporat...
3,Adobe,"[SUSTAINABILITY & SOCIAL IMPACT REPORT2016, L...","[SUSTAINABILITY & SOCIAL IMPACT REPORT2017, D...","[ADOBE CORPORATE SOCIAL , RESPONSIBILITY REPOR...","[12019 , Adobe Corporate Sustainability Report...","[2, 2020 , Adobe Corporate Social Responsibili..."
4,AEScorporation,"[2016 AES SUSTAINABILITY , REPORT, ASPECT: , P...","[AES SUSTAINABILITY REPORT2017, Preliminary ve...","[2018 , SUSTAINABILITY REPORT, 2018 SUSTAINABI...","[2019Sustainability Report2019 , Sustainabilit...","[2020, Performance indicatorsAccelerating the ..."


In [ ]:
data.columns.values[1:]

array(['sentences_2016', 'sentences_2017', 'sentences_2018',
       'sentences_2019', 'sentences_2020'], dtype=object)

In [ ]:
import numpy as np
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

scores = [] #saving the individual company scores
columns = data.columns

# iterating over every company
for i, element in enumerate(data['Company'].values):
  print("Starting report %d"%(i+1))
  comp_scores = []
  for report in data.columns.values[1:]:
    # extracting all sentences from the report
    features = data[data['Company']==element][report].values[0] 
    total_sentences = len(features)

    # variables for saving signal
    signal = 0

    # hitting the model with sentences in chunks to avoid RAM overload
    for batch in chunks(features, 128):
        # calculating softmax for every sentence
        results = tf.nn.softmax(reloaded_model(tf.constant(batch)))
        # argmax to extract label
        argmax = tf.math.argmax(results, axis = 1).numpy()

        #counting pos sentences
        signal += np.count_nonzero(argmax==0)
    # assigning total scores realtive to number of sentences
    comp_scores.append(signal)
  scores.append(comp_scores)

Starting report 1
Starting report 2
Starting report 3
Starting report 4
Starting report 5
Starting report 6
Starting report 7
Starting report 8
Starting report 9
Starting report 10
Starting report 11
Starting report 12
Starting report 13
Starting report 14
Starting report 15
Starting report 16
Starting report 17
Starting report 18
Starting report 19
Starting report 20
Starting report 21
Starting report 22
Starting report 23
Starting report 24
Starting report 25
Starting report 26
Starting report 27
Starting report 28
Starting report 29
Starting report 30
Starting report 31
Starting report 32
Starting report 33
Starting report 34
Starting report 35
Starting report 36
Starting report 37
Starting report 38
Starting report 39
Starting report 40
Starting report 41
Starting report 42
Starting report 43
Starting report 44
Starting report 45
Starting report 46
Starting report 47
Starting report 48
Starting report 49
Starting report 50
Starting report 51
Starting report 52
Starting report 53
St

In [ ]:
scores = np.asarray(scores)
scores = scores.T

data['scores_2016'] = [x for x in scores[0]]
data['scores_2017'] = [x for x in scores[1]]
data['scores_2018'] = [x for x in scores[2]]
data['scores_2019'] = [x for x in scores[3]]
data['scores_2020'] = [x for x in scores[4]]

In [ ]:
data['total_2016'] = data['sentences_2016'].apply(lambda x: len(x))
data['total_2017'] = data['sentences_2017'].apply(lambda x: len(x))
data['total_2018'] = data['sentences_2018'].apply(lambda x: len(x))
data['total_2019'] = data['sentences_2019'].apply(lambda x: len(x))
data['total_2020'] = data['sentences_2020'].apply(lambda x: len(x))

In [ ]:
data

,Company,sentences_2016,sentences_2017,sentences_2018,sentences_2019,sentences_2020,scores_2016,scores_2017,scores_2018,scores_2019,scores_2020,total_2016,total_2017,total_2018,total_2019,total_2020
0,3M,[2016 Sustainability Report “Starting with tec...,"[We are#improvinglives, 90,000 employees helpi...","[2018 , Sustainability Report, We tapped youth...","[Improving every life2019 , S...","[Improvingeverylife2020 , Sustainability Repor...",1745,1535,1806,1809,2096,7537,8099,8215,7996,9147
1,Abbott,"[GLOBALSUSTAINABILITY REPORT 2016, FROM OUR C...","[2017 , GLOBAL , SUSTAINABILITY , REPORT, ABOU...","[GLOBAL SUSTAINABILITY REPORT 2018, CHANGING...","[2019 GLOBAL SUSTAINABILITY REPORTCONTENTS, Su...","[2020 GLOBAL SUSTAINABILIT Y REPORT, COVID-19:...",1335,1913,918,1144,1602,5719,7288,5201,6317,11414
2,Accenture,"[DIFFERENCE, CORPORATE, CITIZENSHIP, REPORT 20...","[CORPORATE, CITIZENSHIPREPORT 2017 , See how S...","[ IMPROVING , THE WAY , THE , WORLD WORKS A...","[BUILDING A FUTURE OF SHARED , SUCCESS, Corp...",[Building a Future of Shared SuccessCorporat...,822,919,802,865,210,2293,2526,1825,2570,435
3,Adobe,"[SUSTAINABILITY & SOCIAL IMPACT REPORT2016, L...","[SUSTAINABILITY & SOCIAL IMPACT REPORT2017, D...","[ADOBE CORPORATE SOCIAL , RESPONSIBILITY REPOR...","[12019 , Adobe Corporate Sustainability Report...","[2, 2020 , Adobe Corporate Social Responsibili...",80,99,124,185,222,409,499,616,818,940
4,AEScorporation,"[2016 AES SUSTAINABILITY , REPORT, ASPECT: , P...","[AES SUSTAINABILITY REPORT2017, Preliminary ve...","[2018 , SUSTAINABILITY REPORT, 2018 SUSTAINABI...","[2019Sustainability Report2019 , Sustainabilit...","[2020, Performance indicatorsAccelerating the ...",2,165,524,518,199,117,1041,2125,3049,18734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,WasteManagemen,"[2017 , Sustainability Report Update, Leading ...","[2018 , Sustainability Report, DRIVING, CHANGE...","[2019 Sustainability Report, Update Summary, D...","[BUILDING VALUE , TOGETHER, The end-of-life f...","[The PeopleBehind Our Progress, WM 2021 Sustai...",74,1351,50,762,601,2580,6691,646,3344,2754
96,WellsFargo,"[W E L L S F , A R G , O & C , O M P A , N ...",[Wells Fargo & Company Corporate Social Respo...,"[Corporate Responsibilty Report 1 , Quis dolup...","[12019, Corporate Responsibility Highlights| ...","[Environmental, Social, and Governance Report...",126,503,475,90,957,384,1596,1280,291,3087
97,Whirlpool,"[2016 SUSTAINABILITY REPORT1, OVERVIEWLetter f...","[2017SUSTAINABILITY REPORTOVERVIEW, Introducti...",[Our Sustainable Journey2018 CORPORATE SUSTAIN...,"[2019 SUSTAINABILITY REPORT, Our Approach .......","[2020 SUSTAINABILITY REPORT, At Whirlpool Corp...",337,432,423,680,1096,1626,1654,2070,3033,3783
98,Wynn,"[2016 , Social Responsibility Report2, Wynn Fa...","[Sustainability Report 2017, A Message from ...",[Care About Everyone and Everything2018 ENVIR...,"[2019 , Environmental, Social, and Governance ...","[2020 , Environmental, Social, and Governance ...",111,467,417,464,611,404,1468,1826,2246,2577


In [ ]:
data[['Company', 'scores_2016','scores_2017','scores_2018','scores_2019','scores_2020','total_2016','total_2017','total_2018','total_2019','total_2020']].to_csv('drive/MyDrive/ml/social_scores.csv')

In [ ]:
for i,element in enumerate(data['scores_2020']):
  print("%s: %.3f"%(data['Company'].values[i],element))

3M: 2096.000
Abbott: 1602.000
Accenture: 210.000
Adobe: 222.000
AEScorporation: 199.000
AlaskaAir: 213.000
Altria: 241.000
Amazon: 939.000
AmericanAirlines: 271.000
Apple: 313.000
AppliedMaterials: 656.000
BD: 1000.000
BestBuy: 375.000
Boeing: 71.000
BostonProperties: 544.000
Caesars: 664.000
Campbells: 694.000
CardinalHealth: 494.000
Carnival: 1229.000
Caterpillar: 387.000
Cigna: 2059.000
Cisco: 1126.000
Colgate: 706.000
Comerica: 804.000
Conagra: 349.000
ConocoPhillips: 1297.000
CSX: 760.000
CVS: 1562.000
DentsplySirona: 820.000
Disney: 862.000
Duke: 429.000
EastmanChemicals: 454.000
Equinix: 270.000
EsteeLauder: 954.000
FMC: 437.000
Goldman: 1076.000
Hasbro: 215.000
Healthpeak: 565.000
HenrySchein: 759.000
HewlettPackard: 900.000
HomeDepot: 1144.000
HormelFood: 600.000
Howmet: 414.000
IBM: 415.000
Intel: 1300.000
InternationalPaper: 1036.000
InterpublicGroup: 786.000
Invesco: 817.000
IronMountain: 556.000
J.M.Smucker: 188.000
Johnson_Johnson: 1839.000
JPMorgan: 675.000
KeyCorp: 584.

In [ ]:
data.head()

,Company,sentences_2016,sentences_2017,sentences_2018,sentences_2019,sentences_2020,scores_2016,scores_2017,scores_2018,scores_2019,scores_2020,total_2016,total_2017,total_2018,total_2019,total_2020
0,ABNAMRO,"[ABN AMRO Group N.V., Integrated Annual Repo...","[ABN AMRO Bank N.V., ABN AMRO Group N.V., Inte...","[Banking for better, for generations to com...","[Accelerating the sustainability , shift, ABN ...","[ABN AMRO’s business, strategy, performance a...",7816,1041,704,964,991,24309,3212,2377,3278,3902
1,Amazon,"[“Jeff, what does Day 2 look like?”, That’s a ...","[2, 0, 1, 7, A N N , U A , LR E P O R T, To ou...","[2, 0, 1, 8, A N N , U A , LR E P O R T, To ou...","[ Sustainability: , Thinking Big September 20...",[All In: Staying the Course on Our Commitment...,1226,1503,1440,341,512,3597,4072,3856,1011,1588
2,asr,"[2016, Annual Report, a.s.r., Archimedeslaan 1...","[2017, Annual Report , ASR Nederland N.V., Ar...","[Annual Report2018, ASR Nederland N.V., Archim...","[Annual Report2019, ASR Nederland N.V., Archim...","[Ditisdetijd, vandoen., Annual Report2020, ASR...",4738,4297,4956,5293,5948,18578,15610,18691,19722,21212
3,bam,"[Integrated ReportIntegrated Report 2015 , Ro...","[Colophon, Royal BAM Group nv, Runnenburg 9, 3...","[Royal BAM Group nv, Runnenburg 9, 3981 , AZ B...","[Integrated Report 2019, BAM plants 150,000 tr...","[Integrated Report 2020, Royal BAM Group nv, R...",4242,4712,4816,5863,4492,10768,12691,12570,14936,11775
4,CocaCola,[Coca‑Cola European Partners plcAnnual Report ...,"[DELIGHTING CUSTOMERS AND CONSUMERS, Coca‑Co...","[GREAT BEVERAGES , GREAT SERVICE , GREAT PEOPL...","[GREAT PEOPLEGREAT SERVICE, GREAT BEVERAGES , ...","[GREAT PEOPLEGREAT SERVICE, GREAT BEVERAGES, 2...",3513,3526,3738,5464,3886,11402,11692,11956,16492,12229


In [ ]:
scores.T

array([[7816, 1041,  704,  964,  991],
       [1226, 1503, 1440,  341,  512],
       [4738, 4297, 4956, 5293, 5948],
       [4242, 4712, 4816, 5863, 4492],
       [3513, 3526, 3738, 5464, 3886],
       [3202, 3392, 3607,  565,  585],
       [2103, 3408, 5706, 5573, 5908],
       [ 387,  400,  417,  490,  418],
       [ 281,  298,  302,  436,  462],
       [2981, 3621, 3832, 3904, 3739],
       [2112, 2312, 2377, 2738, 3129],
       [1964, 2285, 2674, 3579, 3617],
       [9952, 6751, 7345, 7432, 8340],
       [1590, 1171, 1246, 1476, 1803],
       [3438, 3100, 3852, 4089, 4094],
       [ 600,  400,  661,  403,  510],
       [1355, 1569, 1472, 1206, 1391],
       [ 547,  499,  425,  523,  575],
       [5544, 5683, 5353, 5453, 5703],
       [1241, 1735, 1971, 2428, 3815],
       [ 609,  423,  657,  579,  618],
       [ 444,  475,  577,  465,  502],
       [1146, 1254, 1297, 1325, 1468],
       [7058, 6258,  200,  245,  703],
       [7753, 7224, 6141, 5912, 5364]])